<a href="https://colab.research.google.com/github/perlatomdpi/NLP/blob/main/BERT_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using BERT as embedding**
Here we use BERT as an embedding layer and w use it as a vector representation of our words. <br>

This is a techinque that can be use in any personal model: <br>
1 - plug BERT as an embedding layer <br>
2 - put the embedding at the beginning of our model <br>

These steps improve the model --> more powerful and efficient model. <br>



# **Tokenization**
We need to create a BERT layer to have access to meta data for the tokenizer (like vocab size).

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
# we need to add CLS and SEP to be able to use BERT embedding
# that is why we create a funtion for sentence enconding

def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [ ]:
# now we configure all the sentences with the SEP CLS and SEP

data_inputs = [encode_sentence(sentence) for sentence in data_clean]

# **Data creation**
BERT takes 3 differents inputs: <br>
1 - the sentences that we encoded here above <br>
2 - the list of masks and paddng where necessary <br>
3 - the segment input --> 0= first sentence, 1= second sentence --> this process is important to recognize padding tokens.

In [ ]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)                        # 1 - return the id of each token

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)                 # 2 - padding mask: = if it is a padding, 1= if not padding token



def get_segments(tokens):                         # 3 - segment input
    seg_ids = []                                  # create an empty list of our token that will be returned
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":                        # if we passed the first sentence --> switch the token:
            current_seg_id = 1-current_seg_id     # turns 1 into 0 and vice versa --> this recognize padding tokens
    return seg_ids